In [1]:
# Cell 1
!pip install torch torchvision numpy pandas matplotlib openmeteo-requests requests-cache retry-requests

In [28]:
# Cell 2
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry
from datetime import datetime, timezone # <--- Added timezone import

class LiveSensorNetwork:
    """
    Fetches REAL-TIME forecast data from Open-Meteo API instead of historical ERA5.
    """
    def __init__(self):
        # Setup the Open-Meteo client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
        retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
        self.openmeteo = openmeteo_requests.Client(session=retry_session)

    def fetch_current_atmosphere(self, latitude, longitude):
        """
        Fetches the current atmospheric conditions (Steering Layer Winds)
        for a specific location (Lat/Lon) RIGHT NOW.
        """
        url = "https://api.open-meteo.com/v1/forecast"

        # We need wind at 850hPa (Low) and 200hPa (High) for steering flow
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "hourly": ["geopotential_height_500hPa", "wind_speed_850hPa", "wind_direction_850hPa",
                       "wind_speed_200hPa", "wind_direction_200hPa"],
            "timezone": "auto",
            "forecast_days": 1
        }

        try:
            responses = self.openmeteo.weather_api(url, params=params)
            response = responses[0]

            # Process hourly data
            hourly = response.Hourly()

            # Create timezone-aware DatetimeIndex (UTC)
            date_range = pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )

            hourly_data = {
                "time": date_range,
                "z_500": hourly.Variables(0).ValuesAsNumpy(),
                "ws_850": hourly.Variables(1).ValuesAsNumpy(),
                "wd_850": hourly.Variables(2).ValuesAsNumpy(),
                "ws_200": hourly.Variables(3).ValuesAsNumpy(),
                "wd_200": hourly.Variables(4).ValuesAsNumpy()
            }

            df = pd.DataFrame(data=hourly_data)

            # --- FIX: Use timezone-aware 'now' to match DataFrame ---
            now_utc = datetime.now(timezone.utc)

            # Find the row closest to "now"
            # .sub() works now because both are UTC-aware
            closest_idx = df['time'].sub(now_utc).abs().idxmin()
            current_weather = df.iloc[closest_idx]

            print(f"✅ Live Data Fetched for {now_utc.strftime('%Y-%m-%d %H:%M UTC')}:")
            print(f"   Location: {latitude}, {longitude}")
            print(f"   850hPa Wind: {current_weather['ws_850']:.1f} km/h")

            return current_weather

        except Exception as e:
            print(f"❌ API Error: {e}")
            import traceback
            traceback.print_exc()
            return None

# Usage Example
if __name__ == "__main__":
    sensor = LiveSensorNetwork()
    # Fetch for Bay of Bengal (approx center)
    data = sensor.fetch_current_atmosphere(13.08784, 80.27847)

✅ Live Data Fetched for 2025-12-07 05:28 UTC:
   Location: 13.08784, 80.27847
   850hPa Wind: 27.8 km/h


In [29]:
# Cell 3
import numpy as np

class PhysicsEngine:
    def vector_from_speed_dir(self, speed, direction_deg):
        """Converts Speed/Direction to U/V components"""
        rad = np.deg2rad(direction_deg)
        # Met convention: 0 deg is from North (blowing South)
        u = -speed * np.sin(rad)
        v = -speed * np.cos(rad)
        return u, v

    def calculate_realtime_steering(self, weather_data):
        """
        Calculates steering flow using the LIVE data packet.
        """
        # 1. Get components for 850hPa
        u850, v850 = self.vector_from_speed_dir(weather_data['ws_850'], weather_data['wd_850'])

        # 2. Get components for 200hPa
        u200, v200 = self.vector_from_speed_dir(weather_data['ws_200'], weather_data['wd_200'])

        # 3. Velden/Leslie formula (Deep Layer Mean)
        u_steer = (0.75 * u850) + (0.25 * u200)
        v_steer = (0.75 * v850) + (0.25 * v200)

        return u_steer, v_steer

    def detect_stall_risk(self, u_steer, v_steer):
        speed = np.sqrt(u_steer**2 + v_steer**2)
        # If steering wind < 5 km/h, risk of stalling
        return speed < 5.0, speed

In [30]:
# Cell 4
import torch
import torch.nn as nn

class ReSA_ConvLSTM_Cell(nn.Module):
    # (Same architecture as before, omitted for brevity but include the previous class code here)
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        super(ReSA_ConvLSTM_Cell, self).__init__()
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2
        self.conv = nn.Conv2d(input_dim + hidden_dim, 4 * hidden_dim, kernel_size, 1, padding, bias=bias)
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=4)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        combined = torch.cat([input_tensor, h_cur], dim=1)
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)
        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        b, c, h, w = h_next.size()
        flat_h = h_next.view(b, c, -1).permute(2, 0, 1)
        attn_output, _ = self.attention(flat_h, flat_h, flat_h)
        h_next_attn = attn_output.permute(1, 2, 0).view(b, c, h, w)
        return h_next + h_next_attn, c_next

class CycloneTrackCorrector(nn.Module):
    def __init__(self):
        super(CycloneTrackCorrector, self).__init__()
        self.lstm = ReSA_ConvLSTM_Cell(input_dim=3, hidden_dim=64, kernel_size=3, bias=True)
        self.regressor = nn.Linear(64 * 32 * 32, 2)

    def forward(self, x, hidden):
        h, c = self.lstm(x, hidden)
        x_flat = h.view(h.size(0), -1)
        correction = self.regressor(x_flat)
        return correction

    def live_data_to_tensor(self, weather_data):
        """
        Converts the single LIVE data point into a spatial tensor.
        We simulate a field by adding Gaussian noise around the real value.
        """
        # Create base tensor from real values
        # Channel 0: Geopotential, Channel 1: U-wind, Channel 2: V-wind
        real_z = float(weather_data['z_500']) / 100.0 # Normalize roughly
        real_ws = float(weather_data['ws_850'])

        # Create a 32x32 grid filled with the REAL value + some spatial variance
        grid = torch.zeros(1, 3, 32, 32)
        grid[0, 0, :, :] = real_z + torch.randn(32, 32) * 0.1
        grid[0, 1, :, :] = real_ws + torch.randn(32, 32) * 2.0
        grid[0, 2, :, :] = torch.randn(32, 32) * 2.0 # Random V component variation

        return grid

In [31]:
class PhysicsInformedLoss(nn.Module):
    def __init__(self, lambda_phy=0.5):
        super(PhysicsInformedLoss, self).__init__()
        self.mse = nn.MSELoss()
        self.lambda_phy = lambda_phy # Weight of physical reality

    def thermodynamic_limit(self, sst_kelvin):
        """
        Approximation of Maximum Potential Intensity (MPI) based on SST.
        V_max ≈ A * sqrt(SST - 26C)
        """
        sst_c = sst_kelvin - 273.15
        # If SST < 26C, storm decays.
        mpi = torch.where(sst_c > 26.0,
                          80.0 * torch.sqrt(sst_c - 26.0), # Simplified MPI formula
                          torch.zeros_like(sst_c))
        return mpi

    def forward(self, predicted_wind, actual_wind, sst_data):
        # 1. Data Loss (Standard AI learning)
        data_loss = self.mse(predicted_wind, actual_wind)

        # 2. Physics Loss (The PINN part)
        # Calculate theoretical max wind allowed by thermodynamics
        max_possible_wind = self.thermodynamic_limit(sst_data)

        # Penalize if prediction > max_possible_wind (Impossible Physics)
        violation = torch.relu(predicted_wind - max_possible_wind)
        physics_loss = torch.mean(violation ** 2)

        return data_loss + (self.lambda_phy * physics_loss)

In [32]:
class DecisionSupportSystem:
    def __init__(self, cost_matrix):
        """
        cost_matrix = {
            'missed_cyclone': 1000, # Deaths/Disaster (High Penalty)
            'false_alarm': 10       # Economic loss of evacuation (Lower Penalty)
        }
        """
        self.cost_matrix = cost_matrix

    def generate_alert_level(self, probability, predicted_impact_score):
        # Dynamic Thresholding based on Risk
        # If impact is catastrophic, lower the threshold for Red Alert

        risk_score = probability * predicted_impact_score

        if risk_score > 0.8:
            return "RED ALERT: Actionable Threat. Saddle Point Resolved."
        elif risk_score > 0.5:
            # Check Cost-Benefit
            expected_loss_inaction = risk_score * self.cost_matrix['missed_cyclone']
            expected_loss_action = (1 - risk_score) * self.cost_matrix['false_alarm']

            if expected_loss_inaction > expected_loss_action:
                return "ORANGE ALERT: Prepare for evacuation."
            else:
                return "YELLOW ALERT: Monitor. High probability of stall."
        else:
            return "NO ALERT: System likely to dissipate."

# Example Usage
dss = DecisionSupportSystem({'missed_cyclone': 5000, 'false_alarm': 100})
alert = dss.generate_alert_level(probability=0.65, predicted_impact_score=0.9)
print(alert)

ORANGE ALERT: Prepare for evacuation.


In [33]:
# Cell 7
def run_realtime_pipeline():
    # 1. Get Current Location (e.g., Simulated storm center in Bay of Bengal)
    # NOTE: You can change this to any Lat/Lon you want to monitor
    storm_lat, storm_lon = 13.0, 85.0
    now_str = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')

    print(f"--- 🚀 INITIATING AINDRA REAL-TIME SCAN: {now_str} ---")

    # 2. Fetch Live Data
    sensor = LiveSensorNetwork()
    live_data = sensor.fetch_current_atmosphere(storm_lat, storm_lon)

    if live_data is None:
        print("System halted: Could not fetch live weather data.")
        return None, None

    # --- 🔀 BRANCHING LOGIC: CYCLONE GENESIS CHECK ---
    # Threshold: ~30 km/h (17 knots) is typically the start of a Depression
    CYCLONE_THRESHOLD = 30.0
    current_wind_speed = float(live_data['ws_850'])

    # BRANCH 1: NO CYCLONE
    if current_wind_speed < CYCLONE_THRESHOLD:
        print(f"\n--- 🟢 STATUS: No Active Cyclone Detected ---")
        print(f"   Location Monitored: {storm_lat}, {storm_lon}")
        print(f"   🌊 Low-level Wind (850hPa): {current_wind_speed:.2f} km/h")
        print(f"   💨 High-level Wind (200hPa): {live_data['ws_200']:.2f} km/h")
        print(f"   🧭 Wind Direction: {live_data['wd_850']:.0f}°")
        print(f"   ☁️ 500hPa Geopotential Height: {live_data['z_500']:.0f} m")
        print("   ---------------------------------------------")
        print("   Decision: AI Tracking Suspended. Monitoring Mode Active.")
        return None, None # Signal that no tracking is needed

    # BRANCH 2: CYCLONE DETECTED (Proceed with Pipeline)
    else:
        print(f"\n--- 🌪️ WARNING: Active System Detected! (Winds: {current_wind_speed:.1f} km/h) ---")
        print("   -> Initiating Physics Engine...")
        print("   -> Initiating AI Track Correction...")

        # 3. Physics Check
        physics = PhysicsEngine()
        u_steer, v_steer = physics.calculate_realtime_steering(live_data)
        is_stalled, speed = physics.detect_stall_risk(u_steer, v_steer)

        print(f"   Calculated Steering Flow: {speed:.2f} km/h")
        if is_stalled:
            print("   ⚠️ WARNING: Steering currents weak. Saddle Point Detected!")
        else:
            print("   ✅ Steering currents active. Storm moving normally.")

        # 4. AI Correction
        print("--- 🧠 Running AI Correction on Live Data ---")
        model = CycloneTrackCorrector()

        # Process LIVE data into tensor
        input_tensor = model.live_data_to_tensor(live_data)

        # Initialize hidden state
        hidden_state = (torch.randn(1, 64, 32, 32), torch.randn(1, 64, 32, 32))

        # Inference
        correction = model(input_tensor, hidden_state)
        lat_shift = correction[0][0].item()
        lon_shift = correction[0][1].item()

        print(f"   AI Prediction: Storm center will shift by Lat: {lat_shift:.4f}, Lon: {lon_shift:.4f}")

        # 5. Alert
        decider = DecisionSupportSystem({'missed_cyclone': 1000, 'false_alarm': 50})
        impact_score = 0.95 if is_stalled else 0.4
        alert = decider.generate_alert_level(probability=0.8, predicted_impact_score=impact_score)
        print(f"--- 📢 FINAL DECISION: {alert} ---")

        return lat_shift, lon_shift

# Execute
lat_shift, lon_shift = run_realtime_pipeline()

--- 🚀 INITIATING AINDRA REAL-TIME SCAN: 2025-12-07 05:28 UTC ---
✅ Live Data Fetched for 2025-12-07 05:28 UTC:
   Location: 13.0, 85.0
   850hPa Wind: 25.3 km/h

--- 🟢 STATUS: No Active Cyclone Detected ---
   Location Monitored: 13.0, 85.0
   🌊 Low-level Wind (850hPa): 25.34 km/h
   💨 High-level Wind (200hPa): 31.77 km/h
   🧭 Wind Direction: 38°
   ☁️ 500hPa Geopotential Height: 5895 m
   ---------------------------------------------
   Decision: AI Tracking Suspended. Monitoring Mode Active.


In [34]:
# Cell 8
import matplotlib.pyplot as plt

def plot_live_trajectory(lat_shift, lon_shift):
    # Check if we actually have a storm track to plot
    if lat_shift is None or lon_shift is None:
        print("\n🛑 Visualization Skipped: No active cyclone trajectory to plot.")
        return

    start_point = np.array([13.0, 85.0]) # Matches our live fetch location

    # Standard NWP (Linear extrapolation of current steering)
    steps = 5
    nwp_track = [start_point + np.array([0.5 * i, -0.5 * i]) for i in range(steps)]
    nwp_track = np.array(nwp_track)

    # AI Track (Applies the AI correction cumulatively)
    ai_track = [start_point]
    current = start_point.copy()

    real_move = np.array([lat_shift * 0.5, lon_shift * 0.5])

    for i in range(1, steps):
        current = current + np.array([0.5, -0.5]) + (real_move * i)
        ai_track.append(current.copy())

    ai_track = np.array(ai_track)

    # Plot
    plt.figure(figsize=(10, 8))
    ax = plt.gca()
    ax.set_facecolor('#e6f2ff')

    # Chennai
    plt.scatter(80.27, 13.08, color='red', s=150, label='Chennai', marker='*')
    plt.text(80.27, 12.8, "CHENNAI", fontweight='bold')

    # Tracks
    plt.plot(nwp_track[:, 1], nwp_track[:, 0], 'k--', label='Standard Forecast', linewidth=2)
    plt.plot(ai_track[:, 1], ai_track[:, 0], 'g-', label='AINDRA Live Correction', linewidth=3)

    plt.title(f"Live Trajectory Update based on Real-Time Winds", fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Use the variables from the previous cell
plot_live_trajectory(lat_shift, lon_shift)


🛑 Visualization Skipped: No active cyclone trajectory to plot.
